In [93]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import sklearn
import matplotlib.pyplot as plt

In [182]:
data = pd.read_csv("titanic/train.csv")
data.head()
data["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [183]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [184]:
male = data.loc[data["Sex"] == "male"]
female = data.loc[data["Sex"] == "female"]

male_mean_age = male["Age"].mean()
female_mean_age = female["Age"].mean()
male.fillna(male_mean_age, inplace = True)
female.fillna(female_mean_age, inplace = True)

data_f = pd.concat([male,female])
data_f.sort_values(by = ["PassengerId"], inplace = True)


C:\Users\masan\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [185]:
# 生存者の傾向を掴むためにいろいろデータを眺める
# dfference between sex
print(male["Survived"].mean())
print(female["Survived"].mean())

print(pd.crosstab(data_f["Embarked"], data_f["Survived"]))
print(data_f.isnull().sum())

0.18890814558058924
0.7420382165605095
Survived              0    1
Embarked                    
27.915708812260537    0    2
C                    75   93
Q                    47   30
S                   427  217
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [194]:
data_f.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,30.7266,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,27.9157,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,30.7266,S


In [205]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = data_f[["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
X["female?"] = data_f["Sex"].map(lambda x: 1 if x == "female" else 0)
Y = data_f["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state=0, test_size = 0.2)
X_test.head()

C:\Users\masan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Pclass,Age,SibSp,Parch,Fare,Embarked,female?
495,3,30.726645,0,0,14.4583,C,0
648,3,30.726645,0,0,7.5500,S,0
278,3,7.000000,4,1,29.1250,Q,0
31,1,27.915709,1,0,146.5208,C,1
255,3,29.000000,0,2,15.2458,C,1


In [208]:
train_embarked = pd.get_dummies(X_train["Embarked"])
test_embarked = pd.get_dummies(X_test["Embarked"])
train_embarked.drop(27.915708812260537, axis = 1, inplace = True)

In [209]:
X_train.drop(["Embarked"], axis = 1, inplace = True)
X_test.drop(["Embarked"], axis = 1, inplace = True)
X_train_merged = pd.concat([X_train, train_embarked], axis = 1)
X_test_merged = pd.concat([X_test, test_embarked], axis = 1)
X_test_merged.head()

C:\Users\masan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\masan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Pclass,Age,SibSp,Parch,Fare,female?,C,Q,S
495,3,30.726645,0,0,14.4583,0,1,0,0
648,3,30.726645,0,0,7.5500,0,0,0,1
278,3,7.000000,4,1,29.1250,0,0,1,0
31,1,27.915709,1,0,146.5208,1,1,0,0
255,3,29.000000,0,2,15.2458,1,1,0,0


In [200]:
X_train_merged.head()

,Pclass,Age,SibSp,Parch,Fare,female?,C,Q,S
140,3,27.915709,0,2,15.2458,1,1,0,0
439,2,31.000000,0,0,10.5000,0,0,0,1
817,2,31.000000,1,1,37.0042,0,1,0,0
378,3,20.000000,0,0,4.0125,0,1,0,0
491,3,21.000000,0,0,7.2500,0,0,0,1


In [203]:
test_embarked.head()

,C,Q,S
140,1,0,0
439,0,0,1
817,1,0,0
378,1,0,0
491,0,0,1


In [212]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_merged, y_train)
print("train result:", model.score(X_train_merged, y_train))
print("test result:", model.score(X_test_merged, y_test))

train result: 0.797752808988764
test result: 0.7988826815642458


In [292]:
test = pd.read_csv("titanic/test.csv")



mean_fare = test["Fare"].mean()
test["Fare"].fillna(mean_fare, inplace = True)

test_male = test[test["Sex"] == "male"]
test_female = test[test["Sex"] == "female"]

test_male_mean_age = test_male["Age"].mean()
test_female_mean_age = test_female["Age"].mean()
test_male.fillna(test_male_mean_age, inplace = True)
test_female.fillna(test_female_mean_age, inplace = True)

test_f = pd.concat([test_male,test_female])
test_f.sort_values(by = ["PassengerId"], inplace = True)

C:\Users\masan\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [293]:
test_data = test_f[["Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [294]:
test_data_embarked = pd.get_dummies(test_data["Embarked"])
test_data["female?"] = test["Sex"].map(lambda x: 1 if x == "female" else 0)
test_data.drop(["Embarked"], axis = 1, inplace = True)
test_merged = pd.concat([test_data, test_data_embarked], axis = 1)

C:\Users\masan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\masan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [295]:
test_merged.head()

,Pclass,Age,SibSp,Parch,Fare,female?,C,Q,S
0,3,34.5,0,0,7.8292,0,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,1
2,2,62.0,0,0,9.6875,0,0,1,0
3,3,27.0,0,0,8.6625,0,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,1


In [296]:
y = pd.DataFrame({"Survived":model.predict(test_merged)})


In [297]:
test_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          418 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 39.2+ KB


In [298]:
submit = pd.concat([test_f["PassengerId"], y], axis = 1)

In [299]:
submit.set_index("PassengerId", inplace= True)

In [300]:
submit.to_csv("titanic/submit.csv")

In [268]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 1 columns):
Survived    891 non-null int64
dtypes: int64(1)
memory usage: 13.9 KB


In [269]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64